In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
os.chdir("../")
from parc.model import model_burgers as model
import parc.data.data_burgers as data
!nvidia-smi


# Load data

In [ ]:
burgers_data = data.DataBurgers()
R_list = [7500]
a_list = [0.9]
w_list = [0.8]
clipped_data = burgers_data.clip_raw_data(R_list,a_list,w_list, purpose = 'training')

# Training

In [ ]:
# Create tf.dataset
dataset_input = tf.data.Dataset.from_tensor_slices(clipped_data[:,:,:,:3])
dataset_label = tf.data.Dataset.from_tensor_slices(clipped_data[:,:,:,3:])
dataset = tf.data.Dataset.zip((dataset_input, dataset_label))
dataset = dataset.shuffle(buffer_size = 10000) 
dataset = dataset.batch(1)

## Stage 1: Differentiator training

In [ ]:
tf.keras.backend.clear_session()
parc = model.PARCv2_burgers(n_time_step = 1, step_size= 1/100, solver = "heun", mode = "differentiator_training")
#parc.differentiator.load_weights('./Pretrained_Weights/burgers/parc2_diff_burgers_heun.h5')
parc.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00005, beta_1 = 0.9, beta_2 = 0.999))
parc.fit(dataset, epochs = 1, shuffle = True)

In [ ]:
parc.differentiator.save_weights('./Pretrained_Weights/burgers/parc2_TEST_diff_burgers_heun.h5')

## Stage 2: Integrator training

In [ ]:
# Pretrain integrator
tf.keras.backend.clear_session()
parc = model.PARCv2_burgers(n_time_step = 1, step_size= 1/100, solver = "heun", mode = "integrator_training")
parc.differentiator.load_weights('./Pretrained_Weights/burgers/parc2_diff_burgers_heun.h5')
parc.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.00001, beta_1 = 0.9, beta_2 = 0.999))
parc.fit(dataset, epochs = 1, shuffle = True)

In [ ]:
parc.integrator.save_weights('./Pretrained_Weights/burgers/parc2_TEST_int_burgers_heun.h5')